<a href="https://colab.research.google.com/github/EvgeniiaNovolodskaia/LightRAG/blob/main/Small_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb
!pip install -U bitsandbytes
!pip install llama-cpp-python
!pip install rank_bm25 nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
import chromadb
import json
import uuid

from llama_cpp import Llama
from transformers import AutoModelForCausalLM, AutoTokenizer
from jinja2 import Template


from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

import chromadb.utils.embedding_functions as embedding_functions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Large Language Model

## CPU implementation

In [ ]:

llm = Llama.from_pretrained(
    repo_id="bartowski/Llama-3.2-1B-Instruct-GGUF",
    filename="*Q8_0.gguf",
    verbose=False,
    n_ctx=4096,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Llama-3.2-1B-Instruct-Q8_0.gguf:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

llama_new_context_with_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [ ]:
llm.create_chat_completion([{'role':"you give jokes"}])

{'id': 'chatcmpl-9139c4e9-cef8-4f0d-8970-4ed839c70239',
 'object': 'chat.completion',
 'created': 1733827722,
 'model': '/root/.cache/huggingface/hub/models--bartowski--Llama-3.2-1B-Instruct-GGUF/snapshots/067b946cf014b7c697f3654f621d577a3e3afd1c/./Llama-3.2-1B-Instruct-Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "I have a few jokes up my sleeve:\n\n1. Why couldn't the bicycle stand up by itself?\nBecause it was two-tired.\n\n2. What do you call a fake noodle?\nAn impasta.\n\n3. Why did the scarecrow win an award?\nBecause he was outstanding in his field.\n\n4. Why don't scientists trust atoms?\nBecause they make up everything.\n\n5. Why don't eggs tell jokes?\nThey'd crack each other up.\n\n6. Why did the tomato turn red?\nBecause it saw the salad dressing.\n\n7. What do you call a can opener that doesn't work?\nA can't opener.\n\n8. I told my wife she was drawing her eyebrows too high.\nShe looked surprised.\n\n9. Why don't lobsters share?

## GPU Implementation


In [ ]:
# Import model
import torch

model_name = "unsloth/Llama-3.2-1B-Instruct"
#device = "cpu"

def llm_complete(prompt, max_tokens=2048, device="cpu"):

    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    with torch.no_grad():
      ids = tokenizer(prompt, return_tensors='pt').to(device)
      generated_tokens = model.generate(**ids,
                                        max_new_tokens=max_tokens,
                                        )
      answer_only = tokenizer.batch_decode(generated_tokens)[0]

    return answer_only


In [ ]:
prompt = "what is the capital of France?"
llm_complete(prompt,device="cuda")

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'<|begin_of_text|>what is the capital of France? Paris\nThe capital of France is Paris.<|eot_id|>'

## Initial LLM experiments

### Prompt Template
A Jinja2 ``Template`` was created to add the llama 3.2 sepcial tokens in order to optionnaly set roles and wrap a user prompt.

In [ ]:
from jinja2 import Template


my_input = {'role':'you are a clown' ,'input': 'tell me a joke !'}

content = '''\
<|begin_of_text|>\
<|start_header_id|>system<|end_header_id|>\
{{role}}\
<|eot_id|>\
<|start_header_id|>user<|end_header_id|>\
{{input}}\
<|eot_id|>\
<|start_header_id|>assistant<|end_header_id|>\
{{answer}}\
<|eot_id|>
      '''
template = Template(content)
rendered_form = template.render(role=my_input['role'], input=my_input['input'])

rendered_form

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>you are a clown<|eot_id|><|start_header_id|>user<|end_header_id|>tell me a joke !<|eot_id|><|start_header_id|>assistant<|end_header_id|><|eot_id|>\n      '

In [ ]:
import torch
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, falling back to CPU")


Using GPU: Tesla T4


In [ ]:
llm_complete(rendered_form)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>you are a clown<|eot_id|><|start_header_id|>user<|end_header_id|>tell me a joke!<|eot_id|><|start_header_id|>assistant<|end_header_id|><|eot_id|>\n       Why was the math book sad?\nBecause it had too many problems.<|eot_id|>'

In [ ]:
prompt_template = Template(
'''\
<|begin_of_text|>\
<|start_header_id|>system<|end_header_id|>\
{{role}}\
<|eot_id|>\
<|start_header_id|>user<|end_header_id|>\
{{input}}\
<|eot_id|>\
<|start_header_id|>assistant<|end_header_id|>\
{{answer}}\
<|eot_id|>
'''

)

prompt_template = Template(
'''\
<|begin_of_text|>\n
<|start_header_id|>system<|end_header_id|>\n
{{role}}
<|eot_id|>\n
<|start_header_id|>user<|end_header_id|>\n
{{input}}\
<|eot_id|>\n
<|start_header_id|>assistant<|end_header_id|>\n
{{answer}}\
<|eot_id|>
<|end_of_text|>
'''
)

# Retriever

In [ ]:
client = chromadb.Client()
stf_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
collection = client.create_collection(name="warhammer_40k",
                                      metadata={"hnsw:space": "cosine"},
                                      embedding_function=stf_function)

In [ ]:
with open('/content/sample_data/chunks.json', 'r') as f:
    chunks = json.load(f)

{'id': None, 'metadata': {'Header 1': 'Core Rules'}, 'page_content': "'We are beset on all sides by vile predatory aliens and sedition gnaws at us from within; in this dark hour the best we can do is look to our wargear and pray to our gods.'\n- Skolak a'Trellar IV, Imperial Commander  \n200 STORE", 'type': 'Document'}
{'id': None, 'metadata': {'Header 1': 'Introduction'}, 'page_content': '++ THERE IS NO TIME FOR PEACE. NO RESPITE. NO FORGIVENESS. THERE IS ONLY WAR. ++\nWelcome to the Warhammer 40,000 Core Rules! The following pages contain everything you need to know in order to wage glorious battle across the war-torn galaxy of the 41st Millennium. Warhammer 40,000 is a tabletop war game in which players command armies of Citadel miniatures and attempt to defeat their opponent through a mixture of skill, tactics and luck. Storytelling is at the core of Warhammer 40,000, with the rules designed to bring to life the epic conflicts between the forces of Mankind, aliens and daemons in th

In [ ]:
metadatas=[chunk['metadata'] for chunk in chunks]
metadatas[4]

{'Header 1': 'Introduction', 'Header 2': 'The Battle Round (Pg 10-36)'}

In [ ]:
# Add the documents to the collection
collection.add(
    documents=[chunk['page_content'] for chunk in chunks],
    metadatas=[chunk['metadata'] for chunk in chunks],
    ids=[str(uuid.uuid4()) for _ in range(len(chunks))]
)

In [ ]:
question = "What is a visible unit ?"

#Perform a query
results = collection.query(
    query_texts=question,
    n_results=3
)

Create a retrieval function wrapping the ChromaDB query and returning an adapted format.

In [ ]:
def retrieve(question, n_results=5):

  output = {}
  results = collection.query(
        query_texts=question,
        n_results=n_results
    )

  docs = results['documents'][0]
  metas = results['metadatas'][0]

  output = [{'header':metas[i],'text':docs[i]} for i in range(len(docs))]


  return output # retunr texts and associated headers

In [ ]:
retrieve(question)

[{'header': {'Header 1': 'Introduction',
   'Header 2': 'Terrain Features (Pg 44-52)'},
  'text': "- **Model Visible:** If any part of a model can be seen, it is visible.  \n- **Unit Visible:** If any model in a unit is visible, that model's unit is visible.  \n- **Model Fully Visible:** If every Warhammer 40,000 battles are fought across all manner of grim and perilous landscapes, often strewn with ruins, wreckage and other obstacles your forces must navigate while they fight.  \n#### - Unit Fully Visible: If Every Model In A Unit Is Fully Visible, That Unit Is"},
 {'header': {'Header 1': 'Core Concepts',
   'Header 2': 'Determining Visibility'},
  'text': '- **Unit Fully Visible:** If every model in a unit is fully visible, that unit is fully visible.  \n#### Hints And Tips  \nDice Rolling'},
 {'header': {'Header 1': 'Introduction',
   'Header 2': 'Datasheets And Unit Abilities (Pg 37-39)'},
  'text': 'MODEL FULLY VISIBLE\nIf every part of another model that is facing the observing m

## Rag template

A RAG template in Jinja was created

In [ ]:
def prompt_generation(question, chunks):

  rag_template = Template(
  '''\
  <|begin_of_text|>\n
  <|start_header_id|>system<|end_header_id|>\n
  {{ role }}
  <|eot_id|>\n
  <|start_header_id|>user<|end_header_id|>\n
  Answer this question: {{ question }}?
  Use this context:\n
  Context:
  "{% for chunk in chunks %}
  Header: {% for key, value in chunk.header.items() %} - {{ key }}: {{ value }}
  {% endfor %}
  Text: {{ chunk.text }}
  {% endfor %}"
  <|eot_id|>\n
  <|start_header_id|>assistant<|end_header_id|>\n
  '''
  )
  system_role = "You are an experienced wargam player"
  prompt = rag_template.render(**{
    'role': system_role,
    'question': question,
    'chunks':chunks
    })


  return prompt

# Full Rag


In [ ]:
import re
def extract_answer(llm_result):
  # Define regex to match content between tags
  regex = "<\|start_header_id\|>assistant<\|end_header_id\|>([\s\S]*?)<\|eot_id\|>"
  # Extract the content
  matches = re.search(regex, llm_result, re.DOTALL)
  if matches:
    content = matches.group(1)
    return content
  else:
    return "No match found."


In [ ]:
question_0 = "What is a visible unit ?"
question_1 = 'What are the limitations associated to the advance mouvement rule ?'
question_2 = 'Is there a stratagem that can be used to reroll a failed dice role?'
question_3 = 'Explain the Comand Re-roll stratagem'


In [ ]:
def full_rag_cpu(question, n_results=4,device="cpu"):
  retrieved_chunkes = retrieve(question, n_results)
  prompt = prompt_generation(question, retrieved_chunkes)
  llm_answer = llm_complete(prompt,device=device)
  final_answer = extract_answer(llm_answer)
  return final_answer

In [ ]:
final_result = full_rag_cpu(question_2,device="cpu")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
def full_rag_gpu(question, n_results=4,device="cuda"):
  retrieved_chunkes = retrieve(question, n_results)
  prompt = prompt_generation(question, retrieved_chunkes)
  llm_answer = llm_complete(prompt,device=device)
  final_answer = extract_answer(llm_answer)
  return final_answer

In [ ]:
final_result = full_rag_gpu(question_2,device="cuda")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(question_2)
print(final_result)

Is there a stratagem that can be used to reroll a failed dice role?


   In the context of Warhammer 40,000, there is a stratagem that can be used to reroll a failed dice roll: **Using a Cocked Dice**.

   When a dice is rolled and it fails, you can use a cocked dice to reroll the failed dice. This means you can re-roll the entire dice roll, including any modifiers, without having to re-roll individual dice. This can be a useful tactic, especially if you're trying to get a specific result on a dice roll that's unlikely to come up.

   However, there are some caveats to keep in mind:

   1.  **You must re-roll all dice**: You must re-roll all dice involved in the failed roll, including any modifiers.
   2.  **You can't re-roll more than once**: You can never re-roll a dice more than once.
   3.  **You must re-roll before modifiers**: Re-rolls are applied before any modifiers are applied.
   4.  **You can't re-roll a dice more than once**: You must re-roll all dice involved in the failed

# Adding a reranker
The results Might not be satisfactory for some questions.

In order to better use the header extraction, the chunks will be reranked using BM25 over the headers.

In [ ]:
stopwords_en = stopwords.words('english')
stemmer = SnowballStemmer("english")
def metadata_preprocessing(metadata):

  # Tokenize the metadata
  tokens = metadata.split()

  # Remove stopwords and stem the tokens
  filtered_tokens = [stemmer.stem(token.lower()) for token in tokens if token.lower() not in stopwords_en]

  return filtered_tokens

def query_preprocessing(query):

  # Tokenize the query
  query = query.split()

  # Remove stopwords and stem the tokens
  filtered_query = [stemmer.stem(token.lower()) for token in query if token.lower() not in stopwords_en]

  return filtered_query

In [ ]:
def rerank_chunks(question, chunks, n_results=5):
    """
    Re-ranks the given chunks based on their relevance to the question using BM25.

    Args:
        question (str): The input question.
        chunks (list): A list of chunks, each containing a 'header' field with metadata.
        n_results (int): The number of top results to return (default: 5).

    Returns:
        list: Indices of the top `n_results` most relevant chunks.
    """
    # Preprocess headers by joining and processing metadata
    headers = [
        metadata_preprocessing(" ".join(list(chunk['header'].values())))
        for chunk in chunks
    ]

    # Preprocess the query
    query = query_preprocessing(question)

    # Initialize BM25 and calculate scores
    bm25 = BM25Okapi(headers)
    scores = bm25.get_scores(query).argsort()[::-1]  # Sort scores in descending order

    # Return the indices of the top n_results
    return scores.tolist()[:n_results]


## RAG with reranker

In [ ]:
def full_rag_reranker(question, n_results=5):
    """
    Retrieves, reranks, and extracts an answer to a given question using RAG (Retrieval-Augmented Generation).

    Parameters:
        question (str): The input question.
        n_results (int): Number of top results to retrieve and rerank. Default is 5.

    Returns:
        str: The final extracted answer.
    """

    # Step 1: Retrieve relevant chunks based on the input question
    chunks = retrieve(question, n_results=n_results)

    # Step 2: Rerank the retrieved chunks for relevance to the question
    reranked_indices = rerank_chunks(question, chunks, n_results=n_results)
    reranked_chunks = [chunks[i] for i in reranked_indices]

    # Step 3: Generate a prompt using the question and reranked chunks
    my_prompt = prompt_generation(question, reranked_chunks)

    # Step 4: Pass the prompt to the LLM and get a response
    result = llm_complete(my_prompt)

    # Step 5: Extract the final answer from the LLM's response
    final_result = extract_answer(result)

    return final_result

result = full_rag_reranker(question_2, n_results=5)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print("Question:",question_2,"\n")
print("RAG answer:",result)

Question: Is there a stratagem that can be used to reroll a failed dice role? 

RAG answer: 

   As a seasoned wargamer, I can tell you that the answer to your question is yes, there are several stratagems that can be used to reroll a failed dice roll.

In the context of the game, a common strategy is to use the "Roll-Off" rule to reroll a failed dice roll. This involves rolling the dice again, and if the result is higher than the original roll, the player can win the roll-off.

Here are a few examples of how this can be done:

* In the case of the "Deadly Demise" ability, as mentioned in the Core Concepts section of the text, if a model with the Deadly Demise x ability is destroyed, the controlling player rolls one D6 before removing it from play, and if they roll a 6, they can re-roll the D6 to win the roll-off.
* In the "Determining Visibility" section, the text mentions that if a dice is rolled out of bounds, the result can be ignored and rolled again. However, this is not a valid 

# Further application

Implementation with dialog history preservation using [this example.](https://huggingface.co/docs/transformers/conversations).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def format_output(output):

    formatted = (
        "**Here's the answer, buddy:**\n\n"
        f"{output}\n\n"
        "**Pro Tip**: If you need more info, just holler back!"
    )
    return formatted

def full_rag_reranker(question, previous_chat, n_results=5):

    chunks = retrieve(question, n_results=n_results)

    reranked_indices = rerank_chunks(question, chunks, n_results=n_results)
    reranked_chunks = [chunks[i] for i in reranked_indices]

    formatted_chat = [
        {"role": "system", "content": "You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986."}
    ] + previous_chat + [{"role": "user", "content": question}]

    context_text = "\n".join([chunk['text'] for chunk in reranked_chunks])
    formatted_chat.append({"role": "system", "content": f"Context:\n{context_text}"})

    chat_input = tokenizer.apply_chat_template(formatted_chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(chat_input, return_tensors="pt", add_special_tokens=False).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7
        )

    raw_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
    formatted_output = format_output(raw_output)


    previous_chat.append({"role": "user", "content": question})
    previous_chat.append({"role": "assistant", "content": formatted_output})

    return formatted_output, previous_chat

In [ ]:
previous_chat = []

In [ ]:
while True:
  user_query = input("Querr :")
  result_1, previous_chat = full_rag_reranker(user_query, previous_chat)
  print(result_1)

  if user_query == "quit":
    break

Querr :What is visible unit?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Here's the answer, buddy:**

I'm glad we were able to conclude our conversation on a high note. I hope you found the information helpful in understanding the concept of visible units in the grim darkness of the 41st millennium.

If you have any more questions or need further clarification on any of the points we discussed, feel free to ask. Otherwise, I wish you all the best in your gaming endeavors and may your armies be victorious!

**Pro Tip**: Keep in mind that the Warhammer 40,000 universe is vast and complex, and there's always more to learn. If you're new to the game, it's essential to familiarize yourself with the rules and mechanics to maximize your chances of success.

**Pro Tip**: Don't hesitate to reach out if you need any further assistance or guidance. I'm here to help, and I'll do my best to provide you with the information you need to succeed.

Until next time, may your dice rolls be favorable and your armies be victorious!

**Pro Tip**: If you need more info, just ho

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Here's the answer, buddy:**

**Here's a short explanation of each point:**

1. **Damage**: Inflicts equal damage to a model based on its Damage characteristic.
2. **Wounds**: Models lose wounds equal to their Damage characteristic when damaged.
3. **Destroyed**: Models are removed from play if they suffer several wounds and are destroyed.
4. **Attached Units**: Models can merge with other units to form a new unit with a higher Starting Strength.
5. **Move**: The speed at which a model moves across the battlefield.
6. **Toughness**: A model's resilience against physical harm.
7. **Save**: The protection a model's armor gives them.
8. **Wounds**: The damage a model can sustain before being destroyed.
9. **Leadership**: A model's courage, determination, or self-control.
10. **Objective Control**: The model's ability to exert control over an objective on the battlefield.

These points are the core rules of the game, covering damage, wounds, destroyed models, attached units, movement, tou

...

**Here's the answer, buddy:**

**Here's the answer, bro:**

You're Jane, and you've got some awesome Warlord keywords! 

* **Warlord**: You've got the Warlord keyword, which means you're the leader of your army.
* **Faction**: You've chosen a faction for your army, which is great! Now you can use that keyword to customize your army.
* **Enhancements**: You've got enough points to add some cool Enhancements to your army, which will make them even more powerful!
* **Characters**: You can choose one **Character** model to be your Warlord, which will be the leader of your army.
* **Epic Heroes**: You can't have an **Epic Hero** in your army, but you can choose other models to be your Warlord.
* **Transport**: You've got some **Dedicated Transport** units in your army, which is awesome! You can use them to transport your army around the battlefield.

**Pro Tip**: Remember to choose a Warlord that will help you achieve your victory conditions!

**Pro Tip**: If you need more info, just holler

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Here's the answer, buddy:**

It seems like you wanted to end the conversation and didn't continue with the mission. I'll make sure to respect your boundaries.

To confirm, you'd like to:

1. **End The Battle**: This means the battle has ended, and the mission is over.
2. **Determine Victory**: This is the process of figuring out who wins the game. If neither player achieves a victory, the game is considered a draw.
3. **HINTS AND TIPS**: You'd like me to provide some hints and tips to help you with the mission.

I'll try to provide the necessary information to help you with the mission. Please let me know which hints and tips you need.

**Pro Tip**: If you need more info, just holler back!

**Pro Tip**: If you need more info, just holler back!

**Pro Tip**: If you need more info, just holler back!
Querr :quit


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Here's the answer, buddy:**

It seems like the conversation has ended for this session.

To summarize, the mission has concluded, and the following key points have been provided:

* The battle has ended, and the mission is over.
* The next mission will provide additional information and hints to help you with your next challenge.

If you have any further questions or need assistance in the future, please don't hesitate to reach out. Otherwise, I wish you all the best, and may your dice rolls be favorable in the grim darkness of the 41st millennium!

**Pro Tip**: If you need more info, just holler back!
